<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 

In [2]:
import os
from getpass import getpass
import requests
import os
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import json
import gradio as gr
from gradio_calendar import Calendar

def extract_primary_icd10(diagnosis_str):
    try:
        diagnoses = json.loads(diagnosis_str)
        if isinstance(diagnoses, list):
            for d in diagnoses:
                if isinstance(d, dict) and d.get("dx_type") == "PP":
                    return d.get("icd10")
    except (json.JSONDecodeError, TypeError):
        pass
    return None

In [43]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
simb = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/simblist.csv')
disease = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/druglist.csv')
icd10_df = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/icd10 - ICD-10 v.2019.csv')
data['icd10'] = data['diagnosis'].apply(extract_primary_icd10)

In [ ]:
# df = data[data['order_item']!='[]'][['cid','patient','illnes_type','is_follow_up','billing','order_item','total_bill_amount','insurance','diagnosis','icd10','chief_complaint']]


In [6]:
data.head(1)

,id,seq,created_at,updated_at,deleted_at,service_setting,illnes_type,policy_type,cid,cid_type,...,invoice_no,remark,created_submit_by,updated_submit_by,created_submit_at,updated_submit_at,other_insurer,is_check_claim_balance,is_reimbursement,epayment_id
0,5ac01d28-649e-45f1-bd6c-28bef03177fd,1723362821876838256,2024-08-11 14:53:41.876 +0700,2024-09-01 01:03:48.499 +0700,NaN,OPD,ILL,CS,3510600017435,NATIONAL_ID,...,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,N,4f632ac4-ca8e-4f0e-8c8d-7ed56a984f37


In [69]:
disease[disease['ICD-10 Code'].isin(['L720','L028','L029','Z488'])]

,ICD-10 Code,Description
533,L028,L028 ฝีที่ผิวหนัง ตุ่มหนอง และฝึฝักบัวที่ตำแหน...
534,L029,L029 ฝีที่ผิวหนัง ตุ่มหนอง และฝึฝักบัว ไม่ระบุ...
590,L720,L720 ถุงน้ำของหนังกำพร้า (Epidermal cyst)
1331,Z488,Z488 การติดตามดูแลอื่นทางศัลยกรรมที่ระบุรายละเ...


In [167]:
df = data[data['claim_status'].isin(['Approve','Decline'])][['cid','policy_type','created_at','patient','illnes_type','claim_status','response','is_follow_up','billing','order_item','total_bill_amount','insurance','diagnosis','icd10','chief_complaint']]
a = df[df['claim_status']=='Decline'][['cid','policy_type','created_at','patient','claim_status','illnes_type','response','is_follow_up','billing','order_item','total_bill_amount','insurance','diagnosis','icd10','chief_complaint']]


In [166]:
data.columns

Index(['id', 'seq', 'created_at', 'updated_at', 'deleted_at',
       'service_setting', 'illnes_type', 'policy_type', 'cid', 'cid_type',
       'claim_status', 'claim_status_desc', 'ref_id', 'transaction_uid',
       'transaction_no', 'is_follow_up', 'further_claim_id',
       'electronic_signature', 'accident_cause_over45_days', 'admit_date_time',
       'admit_date_time_txt', 'dsc_date_time', 'dsc_date_time_txt',
       'chief_complaint', 'is_procedure', 'icd9', 'dx_free_text', 'coma_score',
       'claim_from_motorcycle', 'visit_with_alcohol_or_drug',
       'alcohol_related', 'addiction_related', 'pregnant', 'total_bill_amount',
       'total_bill_amount_txt', 'patient', 'vital_sign', 'doctor',
       'order_item', 'billing', 'attach_doc_list', 'diagnosis', 'response',
       'check_eligible_id', 'insurance', 'insurance_id', 'hospital_code',
       'hospital_id', 'claim_no', 'discharge_date', 'discharge_date_txt',
       'member_id', 'further_claim', 'patient_registration_id',
    

In [165]:
icd10_df[icd10_df['ICD-10 Code'].isin(['H605','E041','Z098'])]

,One Day Surgery : ODS,3 digit before dot,after dot,ICD-10 Code,Descriptions (EN),Descriptions (TH),illness_type,MTL,Similary,ปล.ในส่วนที่ใช้ ICD10 v.ไม่ใช่ปี2019 ตอนส่งอาจจะมีติด mapping กับประกันบ้าง แต่น้อยมาก
1833,NaN,E04,1.0,E041,Nontoxic single thyroid nodule,คอพอกแบบก้อนเดี่ยวชนิดไม่เป็นพิษ,illness,YES,NaN,NaN
3219,NaN,H60,5.0,H605,"Acute otitis externa, noninfective",หูชั้นนอกอักเสบเฉียบพลัน แบบไม่ติดเชื้อ,illness,YES,NaN,NaN
37785,NaN,Z09,8.0,Z098,Follow-up examination after other treatment fo...,การตรวจติดตามผลหลังการรักษาภาวะอื่นด้วยวิธีอื่น,NaN,YES,NaN,NaN


In [174]:
json.loads(df[df['cid']=='3349900375523']['response'].iloc[0])## คุ้มครองเฉพาะอุบัติเหตุ

{'detail': [],
 'claim_no': 'C029221102',
 'insurer_detail': [{'claim_no': 'C029221102',
   'non_cover': '',
   'benefit_code': 'HSBypass',
   'copay_amount': '',
   'coverage_code': 'ME ไม่สามารถใช้บริการเรียกร้องสินไหมได้ เนื่องจากผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ',
   'excess_amount': '',
   'policy_number': 'P260872803',
   'insurance_type': '',
   'order_sequence': '1',
   'trailer_number': '',
   'approved_amount': '',
   'result_description': 'กรุณาตรวจสอบความคุ้มครองที่เมนู Claim status เวลาทำการ 08.30 – 19.00'},
  {'claim_no': 'C029221102',
   'non_cover': '',
   'benefit_code': 'HSBypass',
   'copay_amount': '',
   'coverage_code': 'ME ไม่สามารถใช้บริการเรียกร้องสินไหมได้ เนื่องจากผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ',
   'excess_amount': '',
   'policy_number': 'P260910639',
   'insurance_type': '',
   'order_sequence': '2',
   'trailer_number': '',
   'approved_amount': '',
   'result_description': 'กรุณาตรวจสอบความคุ้มครองที่เมนู Claim stat

In [147]:
a[a['response'].str.contains('เนื่องจากผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ')]

,cid,created_at,patient,claim_status,illnes_type,response,is_follow_up,billing,order_item,total_bill_amount,insurance,diagnosis,icd10,chief_complaint
12760,3102000763808,2024-08-08 14:46:10.057 +0700,"{""hn"": ""61-21208"", ""dob"": ""1972-01-03"", ""gende...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C028795443"", ""insu...",N,"[{""billing_initial"": ""713.00"", ""billing_discou...","[{""initial"": ""523"", ""item_id"": ""0107E01A0500L0...",1500.0,aia,"[{""icd10"": ""E041"", ""dx_name"": ""Nontoxic single...",E041,คอบวม กลืนน้ำลายแล้วเจ็บ 1 วันก่อนมา รพ.
13803,1959800003819,2024-08-03 11:51:54.284 +0700,"{""hn"": ""2458359"", ""dob"": ""1984-05-12"", ""gender...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C028677108"", ""insu...",N,"[{""billing_initial"": ""1608.00"", ""billing_disco...",[],2758.0,aia,"[{""icd10"": ""M797"", ""dx_name"": ""Fibromyalgia"", ...",M797,ไฟโบรไมอัลเจีย
34969,3100904091957,2024-08-31 13:44:50.206 +0700,"{""hn"": ""048081676"", ""dob"": ""1963-10-06"", ""gend...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029317432"", ""insu...",N,"[{""billing_initial"": ""803.00"", ""billing_discou...",[],905.0,aia,"[{""icd10"": ""J46"", ""dx_name"": ""Status asthmatic...",J46,-
35551,1102800034448,2024-08-27 12:13:28.591 +0700,"{""hn"": ""24-001911"", ""dob"": ""1993-08-10"", ""gend...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029223603"", ""insu...",N,"[{""billing_initial"": ""38"", ""billing_discount"":...","[{""initial"": ""38"", ""item_id"": ""MS0102000012N"",...",3738.0,aia,"[{""icd10"": ""H605"", ""dx_name"": ""Acute otitis ex...",H605,"breast follow up,เสียงดังในหูขวา 2 wk (1,500)"
35641,3349900375523,2024-08-27 10:53:42.787 +0700,"{""hn"": ""23-021650"", ""dob"": ""1969-04-24"", ""gend...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029221102"", ""insu...",N,"[{""billing_initial"": ""150"", ""billing_discount""...","[{""initial"": ""150"", ""item_id"": ""NCF202100001"",...",1800.0,aia,"[{""icd10"": ""Z09.8"", ""dx_name"": ""Follow-up exam...",Z09.8,"ติดตามอาการ +ฟังผลเลือด หลัง D/C,นัดติดตามอาการ"
35648,3809700110921,2024-08-27 12:42:39.845 +0700,"{""hn"": ""10492-56"", ""dob"": ""1978-11-04"", ""gende...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029224391"", ""insu...",N,"[{""billing_initial"": ""9462.00"", ""billing_disco...",[],9210.0,aia,"[{""icd10"": ""N86"", ""dx_name"": ""Erosion and ectr...",N86,ปวดท้องน้อย มีตกขาวสีเหลืองขุ่น ปัสสาวะบ่อย เป...
38351,1103300124287,2024-08-27 20:09:22.660 +0700,"{""hn"": ""01-24-017455"", ""dob"": ""1994-05-25"", ""g...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029237882"", ""insu...",N,"[{""billing_initial"": ""377.78"", ""billing_discou...","[{""initial"": ""377.78"", ""item_id"": ""20341"", ""di...",1500.0,aia,"[{""icd10"": ""N800"", ""dx_name"": ""Endometriosis o...",N800,30yo\n\nlast thin prep 06/2024 NILM \n+HPV ไม่...
41484,1100400391101,2024-08-28 16:14:41.484 +0700,"{""hn"": ""641204821"", ""dob"": ""1989-04-15"", ""gend...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029254581"", ""insu...",N,"[{""billing_initial"": ""2691.00"", ""billing_disco...",[],9321.0,aia,"[{""icd10"": ""I499"", ""dx_name"": ""Cardiac arrhyth...",I499,-
46119,1100400391101,2024-08-29 16:53:08.751 +0700,"{""hn"": ""641204821"", ""dob"": ""1989-04-15"", ""gend...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C029277775"", ""insu...",N,"[{""billing_initial"": ""45.00"", ""billing_discoun...",[],825.0,aia,"[{""icd10"": ""I499"", ""dx_name"": ""Cardiac arrhyth...",I499,หัวใจเต้นผิดจังหวะ
47950,1101499012729,2024-08-04 10:33:20.802 +0700,"{""hn"": ""67-009862"", ""dob"": ""1986-05-01"", ""gend...",Decline,ILL,"{""detail"": [], ""claim_no"": ""C028695836"", ""insu...",N,"[{""billing_initial"": ""9"", ""billing_discount"": ...","[{""initial"": ""9"", ""item_id"": ""04V1V04K0060001""...",2000.0,aia,"[{""icd10"": ""D259"", ""dx_name"": ""Leiomyoma of ut...",D259,นัดมาเพื่อปรึกษา(Consult)Right ovarian cyst G ...


In [132]:
icd10_df[icd10_df['ICD-10 Code'].isin(['K30','I495','J014'])]

,One Day Surgery : ODS,3 digit before dot,after dot,ICD-10 Code,Descriptions (EN),Descriptions (TH),illness_type,MTL,Similary,ปล.ในส่วนที่ใช้ ICD10 v.ไม่ใช่ปี2019 ตอนส่งอาจจะมีติด mapping กับประกันบ้าง แต่น้อยมาก
3525,NaN,I49,5.0,I495,Sick sinus syndrome,กลุ่มอาการซิคไซนัส,illness,YES,NaN,NaN
3754,NaN,J01,4.0,J014,Acute pansinusitis,โพรงอากาศอักเสบเฉียบพลันทุกโพรง,illness,YES,NaN,NaN
4488,NaN,K30,NaN,K30,Functional Dyspepsia,อาหารไม่ย่อย,illness,NO,K300,NaN
